In [1]:
from pathlib import Path
import os
import sys

from dotenv import load_dotenv


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")


ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)
if os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"].strip()

print("ROOT_DIR:", ROOT_DIR)
print("KEY LOADED:", bool(os.getenv("OPENAI_API_KEY")))


ROOT_DIR: /Users/user/Desktop/RAG/backend
KEY LOADED: True


In [ ]:
from app.rag.pipeline import retrieve
from app.rag.router import route_query
from app.rag.vocab.rules import STOPWORDS

import json
import re


def _extract_terms(text: str):
    cleaned = re.sub(r"\s+", " ", text.strip().lower())
    raw_terms = [term for term in cleaned.split(" ") if term]
    stopwords = {word.lower() for word in STOPWORDS}
    terms = []
    for term in raw_terms:
        if term.isdigit() or len(term) < 2:
            continue
        if term in stopwords:
            continue
        terms.append(term)
    seen = set()
    out = []
    for term in terms:
        if term in seen:
            continue
        seen.add(term)
        out.append(term)
    return out


def _collect_keywords(matched: dict, fallback: list[str]):
    keywords = []
    for key in ("card_names", "actions", "payments", "weak_intents"):
        keywords.extend(matched.get(key) or [])
    return keywords or fallback


def _compact_content(text: str, limit: int = 140) -> str:
    if not text:
        return ""
    one_line = " ".join(text.split())
    if len(one_line) <= limit:
        return one_line
    return one_line[:limit] + "..."


def _to_card(doc: dict, keywords: list[str]) -> dict:
    content = doc.get("content") or ""
    doc_id = doc.get("id")
    try:
        doc_id = int(doc_id)
    except (TypeError, ValueError):
        doc_id = 0
    return {
        "id": doc_id,
        "title": doc.get("title") or "",
        "keywords": keywords,
        "content": _compact_content(content),
        "systemPath": "",
        "requiredChecks": [],
        "exceptions": [],
        "regulation": "",
        "detailContent": content,
        "time": "",
        "note": "",
        "relevanceScore": float(doc.get("score") or 0.0),
    }


query = input("질문 입력: ").strip()
if not query:
    raise ValueError("질문을 입력해 주세요.")

routing = route_query(query)
docs = await retrieve(query=query, routing=routing, top_k=4)
fallback_terms = _extract_terms(query)
keywords = _collect_keywords(routing.get("matched") or {}, fallback_terms)

response = {
    "currentSituation": [_to_card(doc, keywords) for doc in docs[:2]],
    "nextStep": [_to_card(doc, keywords) for doc in docs[2:4]],
    "guidanceScript": "",
}

print("던진 질문:",query)
print("route:", routing.get("route"))
print(json.dumps(response, ensure_ascii=False, indent=2))


route: card_usage
{
  "currentSituation": [
    {
      "id": 39,
      "title": "나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요?",
      "keywords": [
        "나라사랑카드",
        "발급"
      ],
      "content": "제목: 나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요? 내용: 나라사랑카드는 일반 체크카드 발급기간보두 보통 3~4일 정도 더 소요되어, 재발급 신청일로부터 약 1주일이 소요됩니다. 그 이유는 의무자의 전자병역증(또는 전역증)이 탑재되는 시간이...",
      "systemPath": "",
      "requiredChecks": [],
      "exceptions": [],
      "regulation": "",
      "detailContent": "제목: 나라사랑카드 재발급 신청 후 수령까지 얼마나 걸리나요?\n\n내용: 나라사랑카드는 일반 체크카드 발급기간보두 보통 3~4일 정도 더 소요되어, 재발급 신청일로부터 약 1주일이 소요됩니다. 그 이유는 의무자의 전자병역증(또는 전역증)이 탑재되는 시간이 소요되기 때문입니다. 입대 시 나라사랑카드를 미소지(분실, 도난 등)인 경우는 육군훈련소, 해군교육사, 공군교육사, 해병대교육훈련단 등에서는 현장 재발급이 가능하오니, 입대 예정일이 2주 이내인 대상자는 미소지한 상태로 입대하여도 무방합니다. (단, 입대 시에는 반드시 입영통지서와 신분증을 지참바랍니다.)",
      "time": "",
      "note": "",
      "relevanceScore": 0.5547207263278539
    },
    {
      "id": 34,
      "title": "나라사랑카드를 신규로 발급받으려면 어떻게 하나요?",
      "keywords": [
        "나라사랑카드",
        "발급"
      ],
     